## 1 数据预处理

### 1.1 数据导入

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('sample_article.txt',header=None)
df.head()

,0,1
0,1,雷锋网按，9月15日百度世界2020大会在线上召开，并由央视新闻全程直播。作为大会的首发环节...
1,2,难不成有生之年还真能一睹外星人真容？在「科学家发现金星有生命存在可能」的热门话题之下，有网友...
2,3,2020年9月5日，由雷锋网 & AI掘金志主办的「第三届中国人工智能安防峰会」在杭州正式召...


In [2]:
df.columns = ['num','text']

In [3]:
df.head()

,num,text
0,1,雷锋网按，9月15日百度世界2020大会在线上召开，并由央视新闻全程直播。作为大会的首发环节...
1,2,难不成有生之年还真能一睹外星人真容？在「科学家发现金星有生命存在可能」的热门话题之下，有网友...
2,3,2020年9月5日，由雷锋网 & AI掘金志主办的「第三届中国人工智能安防峰会」在杭州正式召...


In [4]:
df['text']

0    雷锋网按，9月15日百度世界2020大会在线上召开，并由央视新闻全程直播。作为大会的首发环节...
1    难不成有生之年还真能一睹外星人真容？在「科学家发现金星有生命存在可能」的热门话题之下，有网友...
2    2020年9月5日，由雷锋网 & AI掘金志主办的「第三届中国人工智能安防峰会」在杭州正式召...
Name: text, dtype: object

## 1.2 文本数据分句

In [5]:
from SentenceSplit import cut_sentence_with_quotation_marks
# 分句程序来源：@GLZ1925 https://github.com/GLZ1925/-
sentence_all = []
for item in df['text']:
    sentence_all.append(cut_sentence_with_quotation_marks(item))

In [6]:
len(sentence_all[2])

141

### 1.3 清洗去除停用词

In [7]:
fsw = open('stop_words.txt',encoding='utf-8')
stop_words = []
for line in fsw:
    stop_words.append(line.strip())
stop_words = set(stop_words)

In [8]:
import jieba
for article in sentence_all:
    for i in range(len(article)):
        article[i] = ' '.join(jieba.cut(article[i]))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/vz/l8r_5knx1g3c4yz2yq3mmjnw0000gn/T/jieba.cache
Loading model cost 0.951 seconds.
Prefix dict has been built successfully.


In [13]:
sentence_all[0][0]

'雷锋 网 按 ， 9 月 15 日 百度 世界 2020 大会 在线 上 召开 ， 并 由 央视 新闻 全程 直播 。'

In [11]:
def remove_stopwords(sen):
    sen_new = ' '.join([i.strip() for i in sen if i not in stop_words])
    return sen_new

import jieba

clean_sentence = []
# clean_article=[]

for artricle in sentence_all:
    clean_article=[]
    for i in range(len(artricle)):
        word = remove_stopwords(artricle[i].split())
        clean_article.append(word)
    clean_sentence.append(clean_article)

In [12]:
clean_sentence[0][0]

'雷锋 网 月 15 日 百度 世界 2020 大会 在线 央视 新闻 全程 直播'

### 1.4 加载中文词向量

In [14]:
word_embeddings = {}
f = open('sgns.wiki.word',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    vecs = np.asarray(values[1:],dtype='float32')
    word_embeddings[word] = vecs
f.close()
len(word_embeddings)

352162

### 1.5 生成文本句向量

In [15]:
sentence_vec = []
for item in clean_sentence:
    item_vec = []
    for i in item:
        if len(i) != 0:
            v = sum([word_embeddings.get(w,np.zeros((300,))) for w in i.split()]) / (len(i.split())+0.001)
        else:
            v = np.zeros((300,))
        item_vec.append(v)
    sentence_vec.append(item_vec)

In [16]:
len(sentence_vec[0][0])

300

## 2 获取文章摘要

In [24]:
for num in range(len(sentence_all)):
    '''计算余弦相似度，生成矩阵'''
    sim_mat = np.zeros([len(sentence_all[num]),len(sentence_all[num])])
    from sklearn.metrics.pairwise import cosine_similarity

    for i in range(len(sentence_all[num])):
        for j in range(len(sentence_all[num])):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vec[num][i].reshape(1,300), sentence_vec[num][j].reshape(1,300))[0]
    
    '''TextRank算法计算句子权重排名'''
    import networkx as nx
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    '''提取每篇文章排名最高的两句话作为本文摘要'''
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentence_all[num])), reverse=True)
    print('summary',num)
    print('\n')
    for i in range(2):
        print(scores[i])
        print(''.join(ranked_sentences[i][1].split()))
        print('\n')
    print('\n')

summary 0


0.01051667079604945
从整体来看，前装量产车、AI老司机、5G云代驾这三大要素并非所有玩家都能同时具备，而这也是百度得以一直保持国内自动驾驶


0.011458602229726065
看起来跟一些赛车游戏设备相仿，通过高带宽、低时延的5G网络，云代驾可以从屏幕组上观察汽车周围360°状况，并利用方向盘、档把、脚踏板等控制器驾驶车辆，直到脱困并远程重启无人驾驶系统




summary 1


0.026035994973791696
一项最新进展是：来自美国麻省理工学院、英国曼彻斯特大学和卡迪夫大学的一组科学家借助位于夏威夷的麦克斯韦望远镜（JCMT）和位于智利北方沙漠的ALMA望远镜，在金星大气层中两度发现了生物标记磷化氢（PH3），因此不排除金星存在生命迹象的可能性。


0.03076016940582338
因此，目前科学家们给出了两种可能的解释：存在某种我们未知的光化学或地球化学反应；金星可能存在生命。




summary 2


0.006397392139470044
联邦数据部落，在确保数据安全及用户隐私的前提下，建立基于联邦智能的大数据部落生态，充分发挥各行业参与方的数据价值，推动垂直领域案例落地。


0.007353104628658913
王健宗指出，目前人工智能在移动互联网、云计算、大数据、IoT、5G等新技术的驱动下得以迅猛发展，不过在AI技术落地时总是有所欠缺，即人工智能通用算法在本地化部署过程中所面临的数据困境，而这一块恰恰是相关行业或企业所缺乏的。




